# Attività di EDA su Wikipedia

## Descrizione del dataset

Il dataset offerto è composto da 4 colonne:

- **title**: indica il titolo dell'articolo
- **summary**: contiene l'introduzione dell'articolo
- **documents**: contiene l'articolo completo
- **categoria**: contiene la categoria associata all'articolo

## Obiettivi

### 1. Attività EDA:
È necessario svolgere un'attività di EDA per analizzare e valutare statisticamente tutto il contenuto informativo offerto da Wikipedia. Il dataset fornito possiede le seguenti categorie:

- 'culture'
- 'economics'
- 'energy'
- 'engineering'
- 'finance'
- 'humanities'
- 'medicine'
- 'pets'
- 'politics'
- 'research'
- 'science'
- 'sports'
- 'technology'
- 'trade'
- 'transport'

Per ogni categoria, calcolare le seguenti informazioni:

1. Numero di articoli
2. Numero medio di parole utilizzate
3. Numero massimo di parole presenti nell'articolo più lungo
4. Numero minimo di parole presenti nell'articolo più corto
5. Per ogni categoria, individuare la nuvola di parole più rappresentativa


### 2. Sviluppo classificatore NLP articoli :

Dopo aver svolto l'analisi richiesta, addestrare e testare un classificatore testuale capace di classificare gli articoli (secondo le categorie presenti nel dataset) che saranno in futuro inseriti.


## 1. Attività EDA:

Iniziamo il progetto andandoci a scaricare il dataset di lavoro :

In [0]:
!wget https://proai-datasets.s3.eu-west-3.amazonaws.com/wikipedia.csv

--2024-08-10 14:46:02--  https://proai-datasets.s3.eu-west-3.amazonaws.com/wikipedia.csv
Resolving proai-datasets.s3.eu-west-3.amazonaws.com (proai-datasets.s3.eu-west-3.amazonaws.com)... 3.5.225.173, 52.95.156.56
Connecting to proai-datasets.s3.eu-west-3.amazonaws.com (proai-datasets.s3.eu-west-3.amazonaws.com)|3.5.225.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1003477941 (957M) [text/csv]
Saving to: ‘wikipedia.csv.1’

wikipedia.csv.1     100%[===================>] 956.99M  20.0MB/s    in 68s     

2024-08-10 14:47:11 (14.1 MB/s) - ‘wikipedia.csv.1’ saved [1003477941/1003477941]



Ci predisponiamo il Dataframe spark di lavoro seguendo il seguente codice che va a leggere da un cluster AWS S3 il file csv e lo importa in un dataframe spark.

N.b:
Nel codice che segue, eseguiremo un campionamento stratificato del nostro dataset. Questo approccio è necessario per due ragioni principali poiché in fase di sviluppo stiamo lavorando in un ambiente Databricks Community, pertanto avremo accesso a risorse computazionali limitate.

**Processo di campionamento**

1. Estrarremo un campione del dataset originale.
2. Utilizzeremo un metodo di campionamento stratificato basato sulla colonna "categoria".
3. Questo significa che la distribuzione delle categorie nel nostro campione sarà proporzionalmente la stessa del dataset originale.

Questo approccio ci permetterà di lavorare con un dataset più piccolo e gestibile, mantenendo allo stesso tempo la rappresentatività dei nostri dati originali.

In [0]:

import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('/databricks/driver/wikipedia.csv')
categoria_col = 'categoria'  

# Riduciamo la size per agevolare il running nella versione community di databricks effettuando un campionamento stratificato (in modo che si mantiene bilanciato)
dataset_sample, _ = train_test_split(dataset, test_size=0.1, stratify=dataset[categoria_col], random_state=42)

spark_df_ = spark.createDataFrame(dataset_sample)
spark_df_ = spark_df_.drop("Unnamed: 0")
spark_df_.write.mode("overwrite").saveAsTable("wikipedia")




Mostriamo il contenuto del dataframe spark di lavoro:

In [0]:
spark_df_.show()

+--------------------+--------------------+--------------------+----------+
|               title|             summary|           documents| categoria|
+--------------------+--------------------+--------------------+----------+
|jacques brel metr...|jacques brel is a...|jacques brel is a...| transport|
|     pragya d. yadav|pragya d. yadav (...|pragya d. yadav (...|  research|
|haplochromis sp. ...|haplochromis sp. ...|haplochromis sp. ...|      pets|
|wilhelmina van in...|wilhelmina van in...|wilhelmina van in...|humanities|
|           dash diet|the dash diet (di...|the dash diet (di...|  research|
|charlie van gelderen|charlie van gelde...|charlie van gelde...| economics|
|lockheed l-1049 s...|the lockheed l-10...|the lockheed l-10...|   science|
|1989 nabisco mast...|stefan edberg def...|stefan edberg def...|    sports|
|handheld pc magazine|smartphone & pock...|smartphone & pock...|technology|
|       dancing ballz|dancing ballz is ...|dancing ballz is ...|technology|
|     alumin


Implementeremo adesso una pipeline di Data Cleaning utilizzando SparkNLP. Il processo include:

1. Normalizzazione del testo
2. Lemmatizzazione
3. Rimozione delle stopwords

**Applicazione e Risultati**

- DataFrame "df_cleaned_summary": Versione pulita della colonna "summary" su nuova colonna "cleaned_text"
 

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType


# Per estrarre il testo pulito in una nuova colonna:
from pyspark.sql.functions import concat_ws
import re


# Definiamo la pipeline di Spark NLP che pulisce il testo che agisce su "summary".

# Crea un DocumentAssembler
# Questo componente prende il testo grezzo e lo converte in un documento annotato con metadati di riferimento
# setInputCol("text"): specifica la colonna di input contenente il testo grezzo
# setOutputCol("document"): specifica la colonna di output per il documento annotato
document_assembler = DocumentAssembler().setInputCol("summary").setOutputCol("document")

# Crea un Tokenizer
# Questo componente divide il testo in singole parole o token
# setInputCols(["document"]): specifica la colonna di input (il documento annotato)
# setOutputCol("token"): specifica la colonna di output per i token
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

# Crea un Normalizer
# Questo componente normalizza il testo, ad esempio convertendolo in minuscolo
# setInputCols(["token"]): specifica la colonna di input (i token)
# setOutputCol("normalized"): specifica la colonna di output per i token normalizzati
# setLowercase(True): imposta la conversione in minuscolo
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setCleanupPatterns(["[^\w\s]"]) \
    .setLowercase(True) \
    .setOutputCol("normalized") \
    .setLowercase(True)

# Crea un LemmatizerModel
# Questo componente riduce le parole alla loro forma base (lemma)
# pretrained(): carica un modello pre-addestrato per la lemmatizzazione
# setInputCols(["normalized"]): specifica la colonna di input (i token normalizzati)
# setOutputCol("lemma"): specifica la colonna di output per i lemmi
lemmatizer = LemmatizerModel.pretrained().setInputCols(["normalized"]).setOutputCol("lemma")

# Crea uno StopWordsCleaner
# Questo componente rimuove le parole comuni (stopwords) che spesso non portano significato
# pretrained(): carica una lista pre-definita di stopwords
# setInputCols(["lemma"]): specifica la colonna di input (i lemmi)
# setOutputCol("cleaned"): specifica la colonna di output per le parole pulite
stopwords_cleaner = StopWordsCleaner.pretrained().setInputCols(["lemma"]).setOutputCol("cleaned")

# Crea il pipeline
nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer, 
    normalizer, 
    lemmatizer, 
    stopwords_cleaner
])

# Funzione per applicare la pipeline e pulire il testo
def clean_text_spark_nlp(df, input_col="summary", output_col="cleaned_text"):
    # Fit della pipeline 
    fitted_pipeline = nlp_pipeline.fit(df)
    
    # Applica il pipeline al DataFrame
    result = fitted_pipeline.transform(df)

    # Estraiamo il testo pulito in una nuova colonna
    result_with_clean_text = result.withColumn(output_col, concat_ws(" ", "cleaned.result"))

    # Restituiamo il dataframe con la selezione delle colonne originali e la nuova colonna di testo pulito
    return result_with_clean_text.select(df.columns + [output_col])

# Applichiamo la funzione al tuo DataFrame
df_cleaned_summary = clean_text_spark_nlp(spark_df_)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]


Mostriamo adesso la struttura del df "df_cleaned_summary" con la colonna aggiunta:

In [0]:
df_cleaned_summary.show()

+--------------------+--------------------+--------------------+----------+--------------------+
|               title|             summary|           documents| categoria|        cleaned_text|
+--------------------+--------------------+--------------------+----------+--------------------+
|jacques brel metr...|jacques brel is a...|jacques brel is a...| transport|jacques brel brus...|
|     pragya d. yadav|pragya d. yadav (...|pragya d. yadav (...|  research|pragya yadav bear...|
|haplochromis sp. ...|haplochromis sp. ...|haplochromis sp. ...|      pets|haplochromis sp c...|
|wilhelmina van in...|wilhelmina van in...|wilhelmina van in...|humanities|wilhelmina van in...|
|           dash diet|the dash diet (di...|the dash diet (di...|  research|dash diet dietary...|
|charlie van gelderen|charlie van gelde...|charlie van gelde...| economics|charlie van gelde...|
|lockheed l-1049 s...|the lockheed l-10...|the lockheed l-10...|   science|lockheed l1049 su...|
|1989 nabisco mast...|stefan e


Ora che abbiamo completato la pulizia dei dati, procederemo con il primo step del nostro progetto. Calcoleremo una serie di statistiche importanti per ciascuna categoria presente nel nostro dataset. <br><br> 

Per ogni categoria in particolare andremo a calcolare :

1. Numero totale di articoli  
2. Numero medio di parole per articolo
3. Numero di parole dell'articolo più lungo
4. Numero di parole dell'articolo più corto
5. Nuvola di parole (word cloud) rappresentativa
   Questa visualizzazione mostrerà le parole più frequenti e significative per ciascuna categoria

Queste analisi ci forniranno una panoramica dettagliata della struttura e del contenuto del nostro dataset, evidenziando le differenze tra le varie categorie in termini di lunghezza degli articoli e uso del linguaggio.

### a. Numero di articoli

In [0]:
df_num_articoli = spark.sql("SELECT categoria,COUNT(DISTINCT title) as Num_articoli FROM wikipedia GROUP BY categoria") 
df_num_articoli.show()

+-----------+------------+
|  categoria|Num_articoli|
+-----------+------------+
|    finance|        5303|
|   medicine|        7482|
|   research|        7058|
| technology|        6253|
|     energy|        4393|
|  transport|        7231|
|   politics|         221|
|    culture|        2033|
|    science|        3506|
| humanities|        5477|
|  economics|        3889|
|      trade|        3577|
|     sports|        3515|
|       pets|        4633|
|engineering|        5715|
+-----------+------------+



### b. Numero medio di parole utilizzate

In questa fase, calcoleremo il numero medio di parole utilizzate. <br>
Per eseguire questa analisi, utilizzeremo uno strumento chiamato CountVectorizer.

Il CountVectorizer è uno strumento potente per l'elaborazione del testo 

**Perché usiamo il CountVectorizer?**<br><br>

1. Efficienza: Può gestire grandi volumi di testo in modo efficiente.
2. Versatilità: Ci permette di ottenere facilmente il conteggio delle parole per documento.
3. Preparazione per analisi future: I dati vettorizzati possono essere utilizzati per ulteriori analisi, come la classificazione del testo o il topic modeling.

Nelle prossime celle di codice, definiremo e configureremo il CountVectorizer per la nostra analisi specifica.

In [0]:
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import udf, sum as spark_sum, split
from pyspark.sql.types import IntegerType

# Tokenizziamo la colonna "cleaned_text" 
df_cleaned_summary = df_cleaned_summary.withColumn("words", split(df_cleaned_summary.cleaned_text, "\\s+"))

# Definiamo HashingTF sulla colonna "words"
cv_model_summary = CountVectorizer(inputCol="words", outputCol="word_vector") 



Addestriamo il **CountVectorizer** per il df "df_cleaned_summary":

In [0]:

# Addestro sul df per summary e vettorizzo
cv_model_summary = cv_model_summary.fit(df_cleaned_summary)
df_cleaned_summary_cont_vect = cv_model_summary.transform(df_cleaned_summary)


Andiamo a vedere il tipo di colonna che mi ha generato nel dataframe che abbiamo denominato "word_vector": 

In [0]:
df_cleaned_summary_cont_vect.printSchema()

root
 |-- title: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- documents: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- cleaned_text: string (nullable = false)
 |-- words: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- word_vector: vector (nullable = true)



Dopo aver applicato il CountVectorizer, abbiamo ottenuto una nuova colonna chiamata "word_vector". Questa colonna contiene vettori sparsi che rappresentano il conteggio delle parole per ogni documento. Ecco come procederemo per analizzare questi dati:

1. Comprensione della Colonna "word_vector"
- La colonna "word_vector" è di tipo "vector".
- Ogni elemento di questo vettore rappresenta una parola unica nel vocabolario.
- I valori diversi da zero in questo vettore indicano la presenza e la frequenza di una parola nel documento.

2. Creazione di una Funzione Personalizzata (UDF)<br>
Per contare efficacemente le parole uniche, creeremo una User Defined Function (UDF) che:
- Prende come input il vettore di parole.
- Conta quanti elementi nel vettore sono diversi da zero.
- Restituisce questo conteggio, che rappresenta il numero di parole uniche nel documento.

3. Applicazione della UDF e Calcolo delle Statistiche<br>
Utilizzeremo questa funzione per:
- Contare le parole uniche in ogni articolo.
- Calcolare la media di parole uniche per categoria.

Questo processo ci permetterà di ottenere:
- Una visione dettagliata della ricchezza lessicale di ogni articolo.
- Un'analisi comparativa della diversità di vocabolario tra le diverse categorie.

Nelle prossime celle di codice, implementeremo questa funzione e la applicheremo al nostro dataset.

In [0]:
from pyspark.sql.functions import udf, avg

# Questa funzione è la User Defined Function (UDF) in PySpark che conta il numero di elementi non zero in un vettore sparso
# La funzione viene definita con un decoratore che trasforma la funzione Python in una UDF di Spark.
@udf(returnType=IntegerType())
def count_non_zero(vector):
    return int(vector.numNonzeros()) #è un metodo  per il vettore sparso che restituisce il numero di elementi non zero.

# Calcola il conteggio delle parole uniche (creiamo il campo "unique_words")
df_summary_result_with_count_def = df_cleaned_summary_cont_vect.withColumn("unique_words", count_non_zero("word_vector"))

# Calcolo della media per categoria del n° individuato di parole utilizzate (creiamo il campo "num_medio_words")
df_summary_count_def = df_summary_result_with_count_def.groupBy("categoria").agg(avg("unique_words")).alias("num_medio_words")

Ecco il risultato:

In [0]:
df_summary_count_def.show()

+-----------+------------------+
|  categoria| avg(unique_words)|
+-----------+------------------+
|    finance|49.270867519964995|
|   medicine| 43.68383325981473|
|   research| 42.43473928927267|
| technology| 45.61974466211754|
|     energy| 42.74187126741871|
|  transport| 36.75627947789843|
|   politics| 69.09058892584622|
|    culture| 45.24638457418318|
|    science|47.589231437255755|
| humanities|40.370386452453324|
|  economics| 49.51632047477745|
|      trade|46.033991833131005|
|     sports| 37.13927822536144|
|       pets|31.504489524442967|
|engineering|38.676668841052404|
+-----------+------------------+



### c. Numero massimo di parole su articolo più lungo

Ripartiamo dal df predisposto con il numero di parole per record :

In [0]:
df_summary_result_with_count_def.show()

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|               title|             summary|           documents| categoria|        cleaned_text|               words|         word_vector|unique_words|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|jacques brel metr...|jacques brel is a...|jacques brel is a...| transport|jacques brel brus...|[jacques, brel, b...|(217564,[3,20,21,...|          44|
|     pragya d. yadav|pragya d. yadav (...|pragya d. yadav (...|  research|pragya yadav bear...|[pragya, yadav, b...|(217564,[9,10,18,...|          76|
|haplochromis sp. ...|haplochromis sp. ...|haplochromis sp. ...|      pets|haplochromis sp c...|[haplochromis, sp...|(217564,[41,89,16...|          14|
|wilhelmina van in...|wilhelmina van in...|wilhelmina van in...|humanities|wilhelmina va

Ora procederemo a calcolare il numero massimo di parole utilizzate tra tutti gli articoli per ciascuna categoria. Questo ci darà un'idea della lunghezza dell'articolo più verboso in ogni categoria.

**Procedimento**
1. Utilizzeremo la colonna che abbiamo creato precedentemente, contenente il conteggio delle parole uniche per ogni articolo.
2. Raggrupperemo i dati per categoria.
3. Per ogni gruppo (categoria), troveremo il valore massimo del conteggio di parole.

**Significato del Risultato**
- Questo calcolo ci mostrerà la "diversità lessicale massima" all'interno di ciascuna categoria.
- Ci permetterà di identificare quali categorie tendono ad avere articoli più lunghi o più ricchi di vocabolario.
- Potrebbe rivelare differenze interessanti tra le categorie in termini di complessità o dettaglio degli articoli.

Nelle prossime celle di codice, implementeremo questa analisi utilizzando le funzioni di aggregazione di PySpark.

In [0]:
from pyspark.sql.functions import max

# Calcolo della media per categoria del n° individuato di parole utilizzate 
df_num_max_words_summary = df_summary_result_with_count_def.groupBy("categoria").agg(max("unique_words")).alias("max_words")
df_num_max_words_summary.show()

### d. Numero minimo di parole su articolo più corto


Analogamente per quanto fatto sul massimo numero di parole per categoria, ora ci concentreremo sul calcolo del numero minimo di parole uniche utilizzate per ciascuna categoria. Questa analisi ci permetterà di identificare l' articolo più conciso o sintetico in ogni categoria.

## Procedimento
1. Utilizzeremo la colonna che abbiamo creato precedentemente, contenente il conteggio delle parole uniche per ogni articolo.
2. Raggrupperemo i dati per categoria.
3. Per ogni gruppo (categoria), troveremo il valore minimo del conteggio di parole.

## Significato del Risultato
- Questo calcolo ci mostrerà la "diversità lessicale minima" all'interno di ciascuna categoria.
- Ci permetterà di identificare quali categorie tendono ad avere articoli più brevi o più sintetici.
- Potrebbe rivelare differenze interessanti tra le categorie in termini di concisione o stile di scrittura.
- Potrebbe anche aiutare a identificare potenziali outlier o articoli che potrebbero necessitare di ulteriore elaborazione o verifica.

Nelle prossime celle di codice, implementeremo questa analisi utilizzando le funzioni di aggregazione di PySpark, concentrandoci sulla ricerca del valore minimo per ogni categoria.

In [0]:
from pyspark.sql.functions import min 

# Calcolo della media per categoria del n° individuato di parole utilizzate 
df_num_min_words_summary = df_summary_result_with_count_def.groupBy("categoria").agg(min("unique_words")).alias("min_words")
df_num_min_words_summary.show()

### e. Nuvola parole rappresentativa

Per individuare le parole più significative e rappresentative di ciascuna categoria, utilizzeremo la tecnica TF-IDF (Term Frequency-Inverse Document Frequency). Questa metodologia statistica è ampiamente utilizzata nell'elaborazione del linguaggio naturale e nel recupero delle informazioni.

**Cos'è TF-IDF?**
TF-IDF è una misura statistica che valuta l'importanza di una parola in un documento all'interno di una collezione o corpus. Si compone di due parti:

1. TF (Term Frequency): Misura quanto frequentemente una parola appare in un documento.
2. IDF (Inverse Document Frequency): Misura l'importanza della parola nell'intero corpus.

Il punteggio TF-IDF aumenta proporzionalmente al numero di volte in cui una parola appare nel documento, ma è compensato dalla frequenza della parola nel corpus, il che aiuta a controllare il fatto che alcune parole sono generalmente più comuni di altre.

**Perché usiamo TF-IDF?**

1. Rilevanza: TF-IDF identifica parole che sono importanti in un particolare documento rispetto all'intero corpus.
2. Oggettività: Fornisce una misura statistica dell'importanza delle parole, riducendo la soggettività.
3. Contestualizzazione: Considera non solo la frequenza delle parole, ma anche la loro unicità nel contesto più ampio.

Nelle prossime celle di codice, implementeremo il calcolo TF-IDF e l'estrazione delle parole più rappresentative per ciascuna categoria utilizzando PySpark e le sue librerie di machine learning.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.functions import vector_to_array
from pyspark.ml.feature import IDF

## Addestro sul df per summary e vettorizzo
#cv_model_summary = cv.fit(df_cleaned_summary)
#df_cleaned_summary_cont_vect = cv_model_summary.transform(df_cleaned_summary)


# Esecuzione dell'analisi
vocabulary = cv_model_summary.vocabulary

# Calcolo dell'IDF
idf = IDF(inputCol="word_vector", outputCol="features")
idfModel = idf.fit(df_cleaned_summary_cont_vect)
df_tfidf = idfModel.transform(df_cleaned_summary_cont_vect)


# Convertiamo il vettore TF-IDF in un array
df_tfidf_array = df_tfidf.select(
    "categoria",
    vector_to_array("features").alias("tfidf_array") 
)


Andiamo a verificare il primo output relativo all'implementazione TF-IDF e relativa conversione in array (vedi dataframe df_tfidf_array).

In [0]:
df_tfidf_array.show()

In [0]:

# Creiamo un DataFrame del vocabolario
vocab_df = spark.createDataFrame(
    [(i, word) for i, word in enumerate(vocabulary)], ["index", "word"])

# Esplodiamo l'array TF-IDF andando a determinare il valore per singola word
df_exploded = df_tfidf_array.select(
    "categoria",
    F.explode(F.arrays_zip(
        F.array([F.lit(i) for i in range(len(vocabulary))]), # Definizione dell'indice da 0 a len(vocabulary)-1
        "tfidf_array"                                        # Definizione della singola parola
    )).alias("idx_tfidf")
)


Nel codice precedente abbiamo compiuto due operazioni principali:

1. Definizione del vocabolario:
   Abbiamo creato un "vocabolario" che contiene l'elenco completo di tutte le parole identificate dal **CountVectorizer**. Questo vocabolario ci permette di associare ogni parola a un indice numerico specifico.

2. Creazione del DataFrame "df_exploded":
   Abbiamo generato un nuovo DataFrame chiamato "df_exploded". In questo DataFrame, abbiamo organizzato i valori TF-IDF di ogni parola per ciascuna categoria. Ogni riga di questo DataFrame rappresenta una singola parola in una specifica categoria, con il suo corrispondente valore TF-IDF.

Nelle fasi successive del codice, procederemo come segue:

1. Filtraggio dei risultati:
   Selezioneremo solo i record con un punteggio TF-IDF maggiore di zero. Questo ci permetterà di concentrarci esclusivamente sulle parole che hanno una rilevanza significativa all'interno di ciascuna categoria.

2. Ordinamento e selezione:
   Ordineremo i risultati in base al punteggio TF-IDF, dal più alto al più basso. Questo ci consentirà di identificare le parole più rappresentative per ogni categoria.

3. Visualizzazione dei risultati:
   Mostreremo le prime 5 parole più significative per ogni categoria. Queste parole saranno quelle con i punteggi TF-IDF più elevati, rappresentando così i termini più caratteristici e distintivi di ciascuna categoria.

Questo processo ci permetterà di identificare e visualizzare le parole chiave più rilevanti per ogni categoria, basandoci sulla loro importanza calcolata attraverso il metodo TF-IDF.

Andiamo ora a mostrare il DF "df_exploded" che conterrà la colonna "categoria" e l'elenco dei valori TF-IDF con indice del vocabolario della parola di riferimento:

In [0]:
df_exploded.show()

Andiamo adesso a separare nella colonna "idx_tfidf" l'indice e lo score che utilizzeremo per individuare le parole più rappresentative della categorie:

In [0]:

# Uniamo con il vocabolario e filtriamo i valori positivi
df_words = df_exploded.select(
    "categoria",
    F.col("idx_tfidf.0").alias("index"),
    F.col("idx_tfidf.tfidf_array").alias("tfidf_score")
).filter(F.col("tfidf_score") > 0) \
    .join(vocab_df, "index") \
    .select("categoria", "word", "tfidf_score").distinct()


Ottenuto il DF desiderato andiamo ora per categoria a mostrare le prime 5 parole più rappresentative ordinando in modalità decrescente gli score individuati e prendendo i primi cinque.

In [0]:

# Troviamo le top n parole per ogni categoria
window = Window.partitionBy("categoria").orderBy(F.col("tfidf_score").desc())

df_top_words = df_words.withColumn("rank", F.row_number().over(window)) \
                .filter(F.col("rank") <= 5) \
                .select("categoria", "word", "tfidf_score")

# Visualizzazione dei risultati
print("Top 5 parole più rappresentative per categoria:")
df_top_words.orderBy("categoria", F.col("tfidf_score").desc()).show(truncate=False)